In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/pegasus-my-model.zip /content/
!cp /content/drive/MyDrive/tokenizer.zip /content/
!cp /content/drive/MyDrive/example.pdf /content/
!unzip  pegasus-my-model.zip
!7z x tokenizer.zip
!mv /content/tokenizer/tokenizer.json /content/pegasus-my-model/
!mv /content/tokenizer/tokenizer_config.json /content/pegasus-my-model/

Archive:  pegasus-my-model.zip
   creating: pegasus-my-model/
  inflating: pegasus-my-model/config.json  
  inflating: pegasus-my-model/pytorch_model.bin  
  inflating: pegasus-my-model/generation_config.json  

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 2767662 bytes (2703 KiB)

Extracting archive: tokenizer.zip

WARNINGS:
There are data after the end of archive

--
Path = tokenizer.zip
Type = zip
WARNINGS:
There are data after the end of archive
Physical Size = 2432407
Tail Size = 335255

  0%    Everything is Ok

Archives with Warnings: 1

Warnings: 1
Folders: 1
Files: 4
Size:       8519505
Compressed: 2767662


In [ ]:
!pip install PyPDF2
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00

In [ ]:
from PyPDF2 import PdfReader
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "pegasus-my-model"
tokenizer = AutoTokenizer.from_pretrained("pegasus-my-model")
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
pipe = pipeline('summarization', model = model_ckpt )

In [ ]:
# @title Pdf
# reading pdf-file and breaking it into segments to feed it to model.
reader = PdfReader('example.pdf')
length=len(reader.pages)
segment=[]
Result=[]
for i in range(length):
  page = reader.pages[i]
  text = page.extract_text()
  segment.append(text)

In [ ]:
# @title Pdf input
text=segment[30]
first_part, second_part = text[:len(text)//2], text[len(text)//2:]
pipe_out = pipe(first_part)
Result.append(pipe_out[0]['summary_text'].replace(" .<n>", ".\n"))
pipe_out = pipe(second_part)
Result.append(pipe_out[0]['summary_text'].replace(" .<n>", ".\n"))
print(Result)


['1.The Administrator of the FAA must determine that an emergency exists related to safety in air commerce and issue an immediate order to meet the emergency, with or without notice, under 49 U.S.C. 46105(c). 2.The rescission of a declaration of compliance would go into effect immediately, without the rescission process or the rescission procedures previously described. 3.The FAA would initiate the notification process with the applicant, but would not initiate the rescission process if there is no', '1.The FAA did not receive any comments about the emergency rescission process. 2.An applicant may appeal the decision as provided in 49 U.S.C. 46110 following the issuance of the order. 3.The FAA would consider this petition and issue a final agency decision either affirming or withdrawing the rescission of the declaration of compliance. 4.An applicant would have the opportunity to petition the FAA for reconsideration of a declaration of compliance once a declaration of compliance is resc

In [ ]:
print(first_part,second_part)

4344 Federal Register / Vol. 86, No. 10 / Friday, January 15, 2021 / Rules and Regulations 
57When the Administrator determines that an 
emergency exists related to safety in air commerce and requires immediate action, the Administrator may issue an immediately effective order to meet the emergency, with or without notice, under 49 U.S.C. 46105(c). expressed general support, saying it 
agrees it is important for the FAA to have a mechanism for serious safety issues to be discussed and addressed by the applicant in collaboration with FAA for operations over people. 
The National Association of Mutual 
Insurance Companies (NAMIC) stated publication of the rescission on the FAA website and the Notice of Availability in the Federal Register are insufficient to 
notify pilots that operations over people are no longer safe. NAMIC recommended the FAA use the contact information it has for registered small unmanned aircraft to notify registered operators of safety defects and rescissions. Anot

In [ ]:
#@title Checking 72nd  policy from json file
text="Finally, the NPRM proposed to include radio communication procedures as an area of knowledge covered on the initial aeronautical knowledge test. Several commenters, including Princeton University, Predesa, and Aviation Management, argued that radio communications may not be relevant for all small UAS operations and as such, should be removed from the knowledge test. Predesa suggested that the FAA design a new ‘‘Class G-only unmanned aircraft operator certificate with a small UAS rating’’ that, among other things, does not include radio communication procedures as an area of knowledge that is tested on the knowledge test. One commenter recommended removal of ‘‘radio communication procedures’’ because there is no requirement for radio communications of any sort with small UAS operations. As discussed earlier, the FAA expects that a number of small UAS operationswill take place near an airport. That is why § 107.43 prohibits a small unmanned aircraft from interfering with airport operations or traffic patterns. Understanding radio communication procedures will assist a remote pilot in command operating near a Class G airport in complying with this requirement. Understanding radio communication procedures will assist a remote pilot in command operating near a Class G airport in complying with this requirement if that pilot chooses to use a radio to aid in his or her situational awareness of manned aircraft operating nearby. As described in section 4–1–9 of the Aeronautical Information Manual, manned-aircraft pilots may broadcast their position or intended flight activity or ground operation on the designated Common Traffic Advisory Frequency (CTAF). This procedure is used primarily at airports that do not have an airport traffic control tower, or have a control tower that is not in operation. Pilots of radio-equipped aircraft use standard phraseology to announce their identification, location, altitude, and intended course of action. Self\u0002announcing for arriving aircraft generally begins within 10 nautical miles of the airport and continues until the aircraft is clear of runways and taxiways. Aircraft on the ground intending to depart will begin to make position reports prior to entry of the runway or taxiway and continue until departing the traffic pattern. Aircraft remaining in the pattern make position reports on each leg of the traffic pattern. Thus, knowledge of radio communication procedures will provide a remote pilot in command with the ability to utilize a valuable resource, CTAF, to help determine the position of nearby manned aircraft. As such, this rule will retain this area of knowledge on the initial aeronautical knowledge test."
pipe_out=pipe(text)
print(pipe_out[0]['summary_text'].replace(" .<n>", ".\n"))
#Policy-Extractor gives 88.1% accuracy
#pegasus-my-model gives 95.5% accuracy

1.The NPRM proposed to include radio communication procedures as an area of knowledge covered on the initial aeronautical knowledge test. 2.Several commenters argued that radio communications may not be relevant for all small UAS operations and should be removed from the knowledge test. 3.The FAA expects that a number of small UAS operations will take place near an airport, and understanding radio communication procedures will assist a remote pilot in command operating near a Class G airport in complying with the requirement not to interfere with airport operations or traffic patterns. 4.Knowledge of radio communication procedures will provide a remote pilot in command with the ability to utilize a valuable resource,Knowledge of radio communication procedures, to help determine the position of nearby manned aircraft.


In [ ]:
# @title Importing 2nd model Policy-Extractor
!cp /content/drive/MyDrive/Policy-Extractor.zip /content/
!cp /content/drive/MyDrive/petokenizer.zip /content/
!unzip  Policy-Extractor.zip
!7z x tokenizer.zip -y
!mv /content/tokenizer/tokenizer.json /content/Policy-Extractor/
!mv /content/tokenizer/tokenizer_config.json /content/Policy-Extractor/

Archive:  Policy-Extractor.zip
   creating: Policy-Extractor/
  inflating: Policy-Extractor/pytorch_model.bin  
  inflating: Policy-Extractor/config.json  
  inflating: Policy-Extractor/generation_config.json  

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 2767662 bytes (2703 KiB)

Extracting archive: tokenizer.zip

WARNINGS:
There are data after the end of archive

--
Path = tokenizer.zip
Type = zip
WARNINGS:
There are data after the end of archive
Physical Size = 2432407
Tail Size = 335255

  0%     40% 1 - tokenizer/tokenizer.json                                  46% 1 - tokenizer/tokenizer.json                                 

In [ ]:
from PyPDF2 import PdfReader
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "Policy-Extractor"
tokenizer = AutoTokenizer.from_pretrained("Policy-Extractor")
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
pipe = pipeline('summarization', model = model_ckpt )

In [ ]:
# @title Testing 2nd model Policy-Extractor
text="Finally, the NPRM proposed to include radio communication procedures as an area of knowledge covered on the initial aeronautical knowledge test. Several commenters, including Princeton University, Predesa, and Aviation Management, argued that radio communications may not be relevant for all small UAS operations and as such, should be removed from the knowledge test. Predesa suggested that the FAA design a new ‘‘Class G-only unmanned aircraft operator certificate with a small UAS rating’’ that, among other things, does not include radio communication procedures as an area of knowledge that is tested on the knowledge test. One commenter recommended removal of ‘‘radio communication procedures’’ because there is no requirement for radio communications of any sort with small UAS operations. As discussed earlier, the FAA expects that a number of small UAS operationswill take place near an airport. That is why § 107.43 prohibits a small unmanned aircraft from interfering with airport operations or traffic patterns. Understanding radio communication procedures will assist a remote pilot in command operating near a Class G airport in complying with this requirement. Understanding radio communication procedures will assist a remote pilot in command operating near a Class G airport in complying with this requirement if that pilot chooses to use a radio to aid in his or her situational awareness of manned aircraft operating nearby. As described in section 4–1–9 of the Aeronautical Information Manual, manned-aircraft pilots may broadcast their position or intended flight activity or ground operation on the designated Common Traffic Advisory Frequency (CTAF). This procedure is used primarily at airports that do not have an airport traffic control tower, or have a control tower that is not in operation. Pilots of radio-equipped aircraft use standard phraseology to announce their identification, location, altitude, and intended course of action. Self\u0002announcing for arriving aircraft generally begins within 10 nautical miles of the airport and continues until the aircraft is clear of runways and taxiways. Aircraft on the ground intending to depart will begin to make position reports prior to entry of the runway or taxiway and continue until departing the traffic pattern. Aircraft remaining in the pattern make position reports on each leg of the traffic pattern. Thus, knowledge of radio communication procedures will provide a remote pilot in command with the ability to utilize a valuable resource, CTAF, to help determine the position of nearby manned aircraft. As such, this rule will retain this area of knowledge on the initial aeronautical knowledge test."
pipe_out=pipe(text)
print(pipe_out[0]['summary_text'].replace(" .<n>", ".\n"))

1.The NPRM proposed to include radio communication procedures as an area of knowledge covered on the initial aeronautical knowledge test. 2.Several commenters argued that radio communications may not be relevant for all small UAS operations and should be removed from the knowledge test. 3.The FAA expects that a number of small UAS operations will take place near an airport, and understanding radio communication procedures will assist a remote pilot in command operating near a Class G airport in complying with the requirement not to interfere with airport operations or traffic patterns. 4.Knowledge of radio communication procedures will provide a remote pilot in command with the ability to utilize a valuable resource,’


In [ ]:
 # @title Policy-Extractor gives 88.1% accuracy, pegasus-my-model gives 95.5% accuracy